In [1]:
import sys
print(sys.executable)

C:\Users\chenr\miniforge3\envs\astropy\python.exe


In [ ]:
! dir
# jpt notebook should be started from project root dir

 Volume in drive C has no label.
 Volume Serial Number is 361C-1936

 Directory of C:\Users\chenr\OneDrive\Desktop\phys-134-proj

12/01/2025  09:47 PM    <DIR>          .
12/01/2025  08:11 PM    <DIR>          ..
12/01/2025  09:46 PM            14,748 filesort.ipynb
12/01/2025  09:47 PM    <DIR>          lco_data-20251202-36
12/01/2025  08:20 PM       494,852,926 lco_data-20251202-36.zip
12/01/2025  09:46 PM            16,213 nb.ipynb
12/01/2025  09:34 PM                 0 utcname.ipynb
               4 File(s)    494,883,887 bytes
               3 Dir(s)  112,671,281,152 bytes free


In [11]:
cd lco_data-20251202-36

C:\Users\chenr\OneDrive\Desktop\phys-134-proj\lco_data-20251202-36


In [33]:
! dir

 Volume in drive C has no label.
 Volume Serial Number is 361C-1936

 Directory of C:\Users\chenr\OneDrive\Desktop\phys-134-proj\lco_data-20251202-36

12/01/2025  10:00 PM    <DIR>          .
12/01/2025  09:47 PM    <DIR>          ..
12/01/2025  09:51 PM        13,671,360 2025-11-26-h15-m14-s48-ms233-2025.fits
12/01/2025  09:51 PM        13,671,360 2025-11-26-h15-m14-s48-ms233-coj0m416.fits
12/01/2025  09:51 PM        13,671,360 coj0m416-sq36-20251126-0159-e91.fits
12/01/2025  09:47 PM        13,671,360 coj0m416-sq36-20251126-0159-e91.fits.fz
12/01/2025  09:51 PM        13,795,200 cpt0m439-sq39-20251125-0221-e91.fits
12/01/2025  09:47 PM        13,795,200 cpt0m439-sq39-20251125-0221-e91.fits.fz
12/01/2025  09:51 PM        13,789,440 cpt0m439-sq39-20251126-0207-e91.fits
12/01/2025  09:47 PM        13,789,440 cpt0m439-sq39-20251126-0207-e91.fits.fz
12/01/2025  09:51 PM        13,789,440 cpt0m439-sq39-20251126-0220-e91.fits
12/01/2025  09:47 PM        13,789,440 cpt0m439-sq39-20251126-022

In [35]:
# decompress all from .fits.fz -> .fits
# use astropy
from astropy.io import fits
import shutil, os
for filename in os.listdir('.'):
    if filename.endswith('.fits.fz'):
        fits_filename = filename[:-3]  # Remove the .fz extension
        with fits.open(filename) as hdul:
            hdul.writeto(fits_filename, overwrite=True)
        print(f"Decompressed: {filename} -> {fits_filename}")

Decompressed: coj0m416-sq36-20251126-0159-e91.fits.fz -> coj0m416-sq36-20251126-0159-e91.fits
Decompressed: cpt0m439-sq39-20251125-0221-e91.fits.fz -> cpt0m439-sq39-20251125-0221-e91.fits
Decompressed: cpt0m439-sq39-20251126-0207-e91.fits.fz -> cpt0m439-sq39-20251126-0207-e91.fits
Decompressed: cpt0m439-sq39-20251126-0220-e91.fits.fz -> cpt0m439-sq39-20251126-0220-e91.fits
Decompressed: elp0m414-sq31-20251124-0640-e91.fits.fz -> elp0m414-sq31-20251124-0640-e91.fits
Decompressed: elp0m414-sq31-20251124-0671-e91.fits.fz -> elp0m414-sq31-20251124-0671-e91.fits
Decompressed: elp0m414-sq31-20251124-0700-e91.fits.fz -> elp0m414-sq31-20251124-0700-e91.fits
Decompressed: elp0m414-sq31-20251124-0720-e91.fits.fz -> elp0m414-sq31-20251124-0720-e91.fits
Decompressed: elp0m414-sq31-20251124-0736-e91.fits.fz -> elp0m414-sq31-20251124-0736-e91.fits
Decompressed: elp0m414-sq31-20251125-0662-e91.fits.fz -> elp0m414-sq31-20251125-0662-e91.fits
Decompressed: elp0m414-sq31-20251125-0663-e91.fits.fz -> elp

In [39]:
# lsc0m476-sq34-20251125-0297-e91.fits.fz
# all files are in the formate
# <site><telescope>-<instrucment>XX-<localdate>-<framenum>-<type><reduction>

#rename goal:
# <utcdate>-<site>.fits
def get_utc_date(hdul):
    header = hdul[1].header
    date_obs = header.get('DATE-OBS', '')
    if date_obs:
        #2025-11-26T06:06:06.084
        time_part = date_obs.split('T')
        hh_mm_ssms = time_part[1].split(':')
        utc_date = date_obs.split('T')[0] + '-h' + hh_mm_ssms[0] + '-m' + hh_mm_ssms[1] + '-s' + hh_mm_ssms[2].replace('.', '-ms')
        return utc_date
    return None

#test function
with fits.open('lsc0m476-sq34-20251125-0297-e91.fits') as hdul:
    print(get_utc_date(hdul))  # Expected output: '20251125'

2025-11-26-h06-m06-s06-ms084


In [42]:
# lsc0m476-sq34-20251125-0297-e91.fits.fz
# all files are in the format
# <site><telescope>-<instrucment>XX-<localdate>-<framenum>-<type><reduction>
# telecope is 0m4xx
#rename goal:
# <utcdate>-<site>.fits
for filename in os.listdir('.'):
    if filename.endswith('.fits'):
        with fits.open(filename) as hdul:
            utc_date = get_utc_date(hdul)
            if utc_date:
                site = filename.split('-')[0][:3]  # Extract the site from the filename
                new_filename = f"{utc_date}-{site}.fits"
                # write to new file
                shutil.copyfile(filename, new_filename)
                print(f"New: {filename} -> {new_filename}")
            else:
                print(f"DATE-OBS not found in header of {filename}, skipping.")



New: coj0m416-sq36-20251126-0159-e91.fits -> 2025-11-26-h15-m14-s48-ms233-coj.fits
New: cpt0m439-sq39-20251125-0221-e91.fits -> 2025-11-25-h23-m35-s17-ms210-cpt.fits
New: cpt0m439-sq39-20251126-0207-e91.fits -> 2025-11-26-h23-m00-s14-ms461-cpt.fits
New: cpt0m439-sq39-20251126-0220-e91.fits -> 2025-11-26-h23-m35-s15-ms525-cpt.fits
New: elp0m414-sq31-20251124-0640-e91.fits -> 2025-11-25-h07-m45-s12-ms840-elp.fits
New: elp0m414-sq31-20251124-0671-e91.fits -> 2025-11-25-h08-m30-s09-ms718-elp.fits
New: elp0m414-sq31-20251124-0700-e91.fits -> 2025-11-25-h09-m06-s12-ms841-elp.fits
New: elp0m414-sq31-20251124-0720-e91.fits -> 2025-11-25-h10-m06-s14-ms367-elp.fits
New: elp0m414-sq31-20251124-0736-e91.fits -> 2025-11-25-h11-m06-s18-ms072-elp.fits
New: elp0m414-sq31-20251125-0662-e91.fits -> 2025-11-26-h07-m59-s44-ms591-elp.fits
New: elp0m414-sq31-20251125-0663-e91.fits -> 2025-11-26-h08-m06-s06-ms512-elp.fits
New: elp0m414-sq31-20251126-0631-e91.fits -> 2025-11-27-h07-m44-s42-ms254-elp.fits
New: